# Trabalho II: Algoritmo EM - Inferência Estatística
### Nome: Amanda de Mendonça Perez
### Profº.: Luiz Max de Carvalho

Este notebook buscou implementar o algoritmo EM, tema do segundo trabalho para a disciplina de Inferência Estatística. O material a seguir se propõe a aplicar as fórmulas encontradas (utilizando as bases teóricas desse algoritmo) para o aproximar o estimador de máxima verossimilhança em um problema envolvendo o lançamento de duas moedas cuja probabilidade de se obter cara é diferente. A moeda utilizada para cada sequência de lançamentos é obtida aleatoriamente com probabilidade $50\%$, mas registra-se apenas os resultados de cada lançamento, sem saber qual das moedas foi selecionada. O objetivo do problema é estimar os parâmetros dessa distribuição lidando com a ausência de dados sobre a moeda selecionada a cada rodada. A teoria por trás desse algoritmo foi explicada no trabalho teórico entregue para a obtenção de nota na disciplina.

In [290]:
!python3 --version

Python 3.9.13


In [297]:
import random
import numpy as np

# Simulando o modelo
def escolha_moeda(p1, p2):
    p = random.choices([p1, p2], weights=[0.5,0.5])
    return p

m = 100    # lancamentos de uma mesma moeda
n = 100    # vezes que a moeda é selecionada
p1 = 0.70  # parâmetro da moeda 1
p2 = 0.45  # parâmetro da moeda 2

# Registrando os resultados de cada lançamento
# Note que a moeda selecionada não é registrada
X = np.zeros((n, m))
for i in range(0,n):
    p = escolha_moeda(p1, p2)
    for j in range(0,m):
        X[i,j] = random.choices([0,1], weights = [1-p[0], p[0]])[0]

# Implementando o algoritmo EM para estimar p1 e p2
def k(i, ix, theta, m, s):
    '''
    Retorna o valor da função auxiliar k.
    ---
    Parâmetros:
        i: linha da matriz X sobre a qual estamos iterando;
        ix: índice que indica se estamos calculando p1 (ix=0) ou p2 (ix=1);
        theta: tupla com os valores de p1 e p2 da iteração anterior;
        m: quantidade de colunas de X (i. e., número de lançamentos de uma mesma moeda);
        s: vetor contendo as somas de cada linha de X.  
    '''
    numerador = ((1-theta[ix])**(m-s[i]))*((theta[ix])**(s[i]))
    denominador = ((1-theta[0])**(m-s[i]))*((theta[0])**(s[i])) 
    denominador += ((1-theta[1])**(m-s[i]))*((theta[1])**(s[i]))
    return numerador/denominador

def new_theta(old_theta, m, s):
    '''
    Calcula o novo valor de theta a partir do theta utilizado
    na iteração anterior.
    ---
    Parâmetros:
        old_theta: theta da iteração anterior;
        m: quantidade de colunas de X (i. e., número de lançamentos de uma mesma moeda);
        s: vetor contendo as somas de cada linha de X.
    '''
    num_1, num_2, den_1, den_2 = 0, 0, 0, 0
    for i in range(0,n):
        num_1 += k(i, 0, old_theta, m, s)*s[i]
        num_2 += k(i, 1, old_theta, m, s)*s[i]
        den_1 += k(i, 0, old_theta, m, s)*m
        den_2 += k(i, 1, old_theta, m, s)*m
    return (num_1/den_1, num_2/den_2)  # retorna tupla com estimativa para (p1, p2)

def vinganca_das_moedas(theta_inicial, X, epsilon = 0.001, iterac = 100):
    '''
    Utiliza o algoritmo EM para estimar os parâmetros p1 e p2 que geraram
    os dados originais.
    ---
    Parâmetros:
        theta_inicial: tupla contendo um "chute" inicial para os valores de theta;
        X: matriz com os dados registrados dos lançamentos das moedas;
        epsilon: critério de parada que considera a distância entre dois thetas consecutivos;
        iterac: critério de parada que considera o número máximo de iterações.
    '''
    n = X.shape[0]
    m = X.shape[1]
    somas = np.array([X[i].sum() for i in range(0,n)])
    counter = 0
    while (True):  # garante a entrada no loop
        novo_theta = new_theta(theta_inicial, m, somas)
        counter+=1
        if (((novo_theta[0] - theta_inicial[0])**2 + (novo_theta[1]-theta_inicial[1])**2 < epsilon) \
            or (counter >= 100)): # verifica se algum criterio de parada foi atendido
            break  # caso sim, termina o loop
        theta_inicial = novo_theta  # caso não, atualiza o theta inicial 
    return novo_theta

# Testando com os dados gerados:
estim = vinganca_das_moedas((0.51,0.49), X)
real = (p1, p2)
print(f'Estimativa do algoritmo: {estim};\nParâmetro gerador da simulação: {real}.')

Estimativa do algoritmo: (0.7089242770639296, 0.46180330968832883);
Parâmetro gerador da simulação: (0.7, 0.45).


In [300]:
# Testando com o chute inicial com p1 = p2:
estim = vinganca_das_moedas((0.5,0.5), X)
real = (p1, p2)
print(f'Estimativa do algoritmo: {estim};\nParâmetro gerador da simulação: {real}.')

Estimativa do algoritmo: (0.6019, 0.6019);
Parâmetro gerador da simulação: (0.7, 0.45).
